In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Conv2DTranspose, Layer, Reshape, BatchNormalization, Lambda, Add, Dropout, ReLU, LeakyReLU, Activation, Dropout, Flatten, GaussianNoise
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow import keras
from tensorflow.keras.initializers import HeNormal

# Multi-GPU setup
# strategy = tf.distribute.MirroredStrategy()
# print(f"🔧 Number of devices: {strategy.num_replicas_in_sync}")

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# print(f"🖥️ Total GPU terdeteksi: {len(gpus)}")
# for i, gpu in enumerate(gpus):
#     print(f" - GPU {i}: {gpu}")

In [ ]:
IMG_SIZE = (128, 128)
BATCH = 32
DIR = '/kaggle/input/dataset-batik-campuran'
DEST_PATH = '/kaggle/working/batik_campuran'

In [ ]:
import os
import shutil


shutil.copytree(DIR, DEST_PATH)

# Step 2: Rename semua file di folder itu
for filename in os.listdir(DEST_PATH):
    new_filename = (
        filename.replace(" ", "_")
                .replace("(", "")
                .replace(")", "")
                .replace("'", "")
    )
    old_path = os.path.join(DEST_PATH, filename)
    new_path = os.path.join(DEST_PATH, new_filename)

    if old_path != new_path:
        print(f"Renaming: {filename} -> {new_filename}")
        os.rename(old_path, new_path)

In [ ]:
from pathlib import Path

folder = Path("/kaggle/working/batik_campuran/batik_campuran")

for file in folder.iterdir():
    if file.is_file():
        new_name = (
            file.name.replace(" ", "_")
                     .replace("(", "")
                     .replace(")", "")
        )
        new_path = file.parent / new_name
        if file.name != new_name:
            print(f"Renaming: {file.name} -> {new_name}")
            file.rename(new_path)


In [ ]:
import os
import imghdr
from PIL import Image
import shutil

SRC = "/kaggle/working/batik_campuran/batik_campuran"
DST = "/kaggle/working/batik_campuran_valid"
os.makedirs(DST, exist_ok=True)

for filename in os.listdir(SRC):
    src_path = os.path.join(SRC, filename)
    dst_path = os.path.join(DST, filename)

    if os.path.isfile(src_path):
        try:
            # pastikan benar-benar file image
            img_type = imghdr.what(src_path)
            if img_type in ["jpeg", "png", "bmp", "gif"]:
                with Image.open(src_path) as img:
                    img.verify()
                shutil.copy2(src_path, dst_path)
            else:
                print(f"Skipping (not image): {filename} — Detected as {img_type}")
        except Exception as e:
            print(f"Skipping (error): {filename} — {e}")


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img, array_to_img
import random

# Path ke folder gambar asli
folder_path = DST

# Augmentasi ringan
def random_augment(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    
    # Rotate 0, 90, 180, atau 270 derajat
    k = random.choice([0, 1, 2, 3])
    image = tf.image.rot90(image, k=k)
    
    return image

# Loop semua file di folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(folder_path, filename)
        image = load_img(image_path)
        image = img_to_array(image)

        # Resize kalau perlu (misalnya resize ke 128x128)
        image = tf.image.resize(image, [128, 128])
        image = tf.cast(image, tf.uint8)

        # Augment 3x per gambar
        for i in range(3):
            aug_img = random_augment(image)
            aug_img = tf.clip_by_value(aug_img, 0, 255)
            aug_img = array_to_img(aug_img)

            new_filename = filename.split('.')[0] + f'_aug{i}.jpg'
            aug_img.save(os.path.join(folder_path, new_filename))

print("Augmentasi selesai 🚀")


In [ ]:
dataset = image_dataset_from_directory (
    "/kaggle/working/batik_campuran_valid",
    batch_size=BATCH,
    image_size=IMG_SIZE,
    class_names=None,
    label_mode=None
)

In [ ]:
def preprocess(image):
    img = tf.image.resize(image, [128, 128], method='bicubic')
    img = tf.cast(img, tf.float32) / 127.5 - 1.0
    return img

dataset = dataset.map(preprocess).shuffle(500).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# def color_augment(x):
#     x = tf.image.random_brightness(x, max_delta=0.1)
#     x = tf.image.random_saturation(x, lower=0.8, upper=1.2)
#     return tf.clip_by_value(x, 0.0, 1.0)

# def translation_augment(x):
#     img_shape = tf.shape(x)
#     shift_x = 


# def diff_augment(x, policy='color, translation'):
#     if 'color' in policy:
#         x = color_augmentx = tf.image.random_brightness(x, max_delta=0.1)
#     x = tf.image.random_saturation(x, lower=0.8, upper=1.2)
#     return tf.clip_by_value(x, 0.0, 1.0)

In [ ]:
def translation_augment(x, ratio=0.02):
    shape = tf.shape(x)
    shift_x = tf.cast(ratio * tf.cast(shape[1], tf.float32), tf.int32)
    shift_y = tf.cast(ratio * tf.cast(shape[2], tf.float32), tf.int32)

    dx = tf.random.uniform([], -shift_x, shift_x + 1, dtype=tf.int32)
    dy = tf.random.uniform([], -shift_y, shift_y + 1, dtype=tf.int32)

    return tf.roll(x, shift=[dx, dy], axis=[1, 2])


def diff_augment(x, policy='translation'):
    if 'translation' in policy:
        x = translation_augment(x)
    return x


In [ ]:
# Ambil satu batch dari dataset
for batch in dataset.take(1):
    batch_image = batch.numpy()  # kalau label ikut, bisa unpack: batch_image, _ = batch
    break

# Rescale dari [-1, 1] ke [0, 255]
batch_image = ((batch_image + 1) * 127.5).astype(np.uint8)

# Plot gambar
plt.figure(figsize=(10, 10))
for i in range(min(5, len(batch_image))):
    plt.subplot(1, 5, i + 1)
    plt.imshow(batch_image[i])
    plt.axis('off')

plt.show()

In [ ]:
# !ls /kaggle/working/batik_campuran/batik_campuran -R

# import os

# dst_folder = "/kaggle/working/batik_campuran/batik_campuran"

# for filename in os.listdir(dst_folder):
#     print(repr(filename))

# print(sorted(os.listdir("/kaggle/working/batik_campuran/batik_campuran")))

In [ ]:
# KI = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
# input_dim = 128

# def build_generator():
#     Generator = Sequential()

#     Generator.add(Dense(8 * 8 * 512, input_dim = input_dim))
#     Generator.add(ReLU())
    
#     Generator.add(Reshape((8, 8, 512)))

#     # karena input mulai dari 8 berati ini 16x16x256 karena stridesnya 2 jadi nanti dibikin 2x lipat 256->128
#     Generator.add(Conv2DTranspose(256, kernel_size=4, strides=2, padding='same'))
#     Generator.add(BatchNormalization())
#     Generator.add(ReLU())

#     # Nah ini udah 32x32x128
#     Generator.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
#     Generator.add(BatchNormalization())
#     Generator.add(ReLU())

#     # Sekarang 64x64x64
#     Generator.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
#     Generator.add(BatchNormalization())
#     Generator.add(ReLU())

#     # Sekarang 128x128x32
#     Generator.add(Conv2DTranspose(32, kernel_size=4, strides=2, padding='same'))
#     Generator.add(BatchNormalization())
#     Generator.add(ReLU())

#     # Sekarang 256x256x3. Nah disini udh sampe ke 256 berarti udah sama kayak image_size nya. 
#     # Disini kenapa 32 langsung lonncat 3 itu gpp karena disini 3 merejuk ke channel gambar RGB
#     # Disini output layers, tetep pake Tranpose karena generator itu buat bikin gambar jadi butuh upsampling bukan down
#     Generator.add(Conv2DTranpose(3, kernel_size=4, strides=2, padding='same'))
#     Generator.add(Activation('tanh'))

#     return Generator

# generator = build_generator()
# generator.summary()

# keras.utils.plot_model(generator, show_shapes=True)

In [ ]:


KI = HeNormal()

class ResidualBlock(Layer):
    def __init__(self, filters, **kwargs):
        super(ResidualBlock, self).__init__(**kwargs)
        self.conv1 = Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer=KI)
        self.bn1 = BatchNormalization()
        self.relu1 = ReLU()
        self.conv2 = Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer=KI)
        self.bn2 = BatchNormalization()
        self.add = Add()
        self.relu2 = ReLU()

    def call(self, x):
        skip = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.add([x, skip])
        x = self.relu2(x)
        return x


In [ ]:
KI = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
input_dim = 256

def build_generator():
    Generator = Sequential()

    Generator.add(Dense(4 * 4 * 1024, input_dim = input_dim))
    Generator.add(LeakyReLU(alpha=0.2))
    
    Generator.add(Reshape((4, 4, 1024)))
    Generator.add(GaussianNoise(0.1))

    # karena input mulai dari 4 berati ini 8x8x512 karena stridesnya 2 jadi nanti dibikin 2x lipat 256->128
    Generator.add(Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', kernel_initializer=KI))
    Generator.add(BatchNormalization())
    Generator.add(LeakyReLU(alpha=0.2))

    # Nah ini udah 16x16x256
    Generator.add(Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', kernel_initializer=KI))
    Generator.add(BatchNormalization())
    Generator.add(LeakyReLU(alpha=0.2))

    # Sekarang 32x32x128
    Generator.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', kernel_initializer=KI))
    Generator.add(BatchNormalization())
    Generator.add(LeakyReLU(alpha=0.2))

    Generator.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', kernel_initializer=KI))
    Generator.add(BatchNormalization())
    Generator.add(LeakyReLU(alpha=0.2))

    Generator.add(Conv2DTranspose(64, kernel_size=4, strides=1, padding='same', kernel_initializer=KI))
    Generator.add(BatchNormalization())
    Generator.add(LeakyReLU(alpha=0.2))

    Generator.add(ResidualBlock(64))
    Generator.add(ResidualBlock(64))

    # Sekarang 256x256x3. Nah disini udh sampe ke 256 berarti udah sama kayak image_size nya. 
    # Disini kenapa 32 langsung lonncat 3 itu gpp karena disini 3 merejuk ke channel gambar RGB
    # Disini output layers, tetep pake Tranpose karena generator itu buat bikin gambar jadi butuh upsampling bukan downsampling
    Generator.add(Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', kernel_initializer=KI))
    Generator.add(Activation('tanh'))

    return Generator

generator = build_generator()
generator.summary()

keras.utils.plot_model(generator, show_shapes=True)

In [ ]:
def build_discriminator():
    input_shape = (128, 128, 3)

    Discriminator = Sequential()
    
    Discriminator.add(Conv2D(32, kernel_size=4, strides=2, padding='same', input_shape=input_shape))
    Discriminator.add(LeakyReLU(alpha=0.2))
    Discriminator.add(Dropout(0.2))
    

    Discriminator.add(Conv2D(64, kernel_size=4, strides=2, padding='same'))
    Discriminator.add(LeakyReLU(alpha=0.2))
    Discriminator.add(Dropout(0.2))
    

    Discriminator.add(Conv2D(128, kernel_size=4, strides=2, padding='same'))
    Discriminator.add(LeakyReLU(alpha=0.2))
    Discriminator.add(Dropout(0.2))
    


    Discriminator.add(Flatten())
    
    Discriminator.add(Dense(128))
    Discriminator.add(LeakyReLU())
    Discriminator.add(Dense(1, activation='sigmoid'))

    return Discriminator

discriminator = build_discriminator()
discriminator.summary()
keras.utils.plot_model(discriminator, show_shapes=True, show_dtype=True)

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import zipfile

SAVE_DIR = "/kaggle/working/GAN_Batik"
MODEL_DIR = os.path.join(SAVE_DIR, "models")
IMG_DIR = os.path.join(SAVE_DIR, "images")

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(IMG_DIR, exist_ok=True)

# Fungsi buat save gambar
def save_generated_images(generator, latent_dim, epoch, save_dir=IMG_DIR, n=5):
    noise = tf.random.normal([n * n, latent_dim])
    generated = generator(noise, training=False)
    generated = (generated + 1) / 2.0  # scale ke [0, 1]

    fig, axes = plt.subplots(n, n, figsize=(n, n))
    idx = 0
    for i in range(n):
        for j in range(n):
            axes[i][j].imshow(generated[idx])
            axes[i][j].axis("off")
            idx += 1
    plt.tight_layout()
    filepath = os.path.join(save_dir, f"epoch_{epoch+1}.png")
    plt.savefig(filepath)
    plt.close()

# Fungsi buat save model
def save_models(generator, discriminator, epoch, save_dir=MODEL_DIR):
    gen_base = os.path.join(save_dir, f"generator_epoch_{epoch+1}")
    disc_base = os.path.join(save_dir, f"discriminator_epoch_{epoch+1}")

    # Simpan dalam format .keras (format baru, lebih aman buat custom layer)
    generator.save(gen_base + ".keras")
    discriminator.save(disc_base + ".keras")

    # Simpan dalam format .h5 (lebih umum, tapi hati-hati kalo ada Lambda/custom layer)
    generator.save(gen_base + ".h5")
    discriminator.save(disc_base + ".h5")

    # Simpan hanya weight-nya (.weights.h5)
    generator.save_weights(gen_base + ".weights.h5")
    discriminator.save_weights(disc_base + ".weights.h5")

# Fungsi auto-zip setelah training selesai
def zip_output(zip_path="/kaggle/working/GAN_Batik.zip", folder_to_zip=SAVE_DIR):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_to_zip):
            for file in files:
                filepath = os.path.join(root, file)
                arcname = os.path.relpath(filepath, folder_to_zip)
                zipf.write(filepath, arcname)


def load_models(epoch, model_dir=MODEL_DIR):
    gen_path = os.path.join(model_dir, f"generator_epoch_{epoch}.keras")
    disc_path = os.path.join(model_dir, f"discriminator_epoch_{epoch}.keras")

    generator = tf.keras.models.load_model(gen_path, compile=False)
    discriminator = tf.keras.models.load_model(disc_path, compile=False)
    return generator, discriminator


In [ ]:
def generate_and_plot_images(generator, latent_dim, n=5):
    noise = tf.random.normal([n * n, latent_dim])
    generated = generator(noise, training=False)
    generated = generated.numpy()

    fig, axes = plt.subplots(n, n, figsize=(n, n))
    for i in range(n):
        for j in range(n):
            axes[i][j].imshow(generated[i * n + j])
            axes[i][j].axis("off")
    plt.tight_layout()
    plt.show()

def train_gan(generator, discriminator, dataset, latent_dim, g_optimizer, d_optimizer, loss_fn, epochs):
    max_steps_per_epoch = 243  # Boleh disesuaikan

    for epoch in range(epochs):
        print(f"\n🌟 Epoch {epoch+1}/{epochs} mulai...")
        total_steps = min(len(dataset), max_steps_per_epoch)

        for step, real_images in enumerate(dataset):
            if step >= max_steps_per_epoch:
                break
                
            real_images = tf.convert_to_tensor(real_images)
            batch_size = real_images.shape[0]
            random_noise = tf.random.normal(shape=(batch_size, latent_dim))

            # ----------- Discriminator Training -----------
            with tf.GradientTape() as tape:
                # Tambah sedikit noise ke gambar real & fake (stabilin training)
                real_images_noisy = real_images + tf.random.normal(tf.shape(real_images), mean=0.0, stddev=0.05)
                fake_images = generator(random_noise, training=True)
                fake_images_noisy = fake_images + tf.random.normal(tf.shape(fake_images), mean=0.0, stddev=0.05)

                aug_real = diff_augment(real_images_noisy)
                aug_fake = diff_augment(fake_images_noisy)
                
                pred_real = discriminator(aug_real, training=True)
                pred_fake = discriminator(aug_fake, training=True)

                real_labels = tf.ones_like(pred_real) * 0.9  # Label smoothing
                fake_labels = tf.zeros_like(pred_fake)

                d_loss_real = loss_fn(real_labels, pred_real)
                d_loss_fake = loss_fn(fake_labels, pred_fake)
                d_loss = (d_loss_real + d_loss_fake) / 2

            gradients_d = tape.gradient(d_loss, discriminator.trainable_variables)
            d_optimizer.apply_gradients(zip(gradients_d, discriminator.trainable_variables))

            # ----------- Generator Training -----------
            with tf.GradientTape() as tape:
                fake_images = generator(random_noise, training=True)
                pred_fake = discriminator(fake_images, training=True)
                g_loss = loss_fn(tf.ones_like(pred_fake), pred_fake)

            gradients_g = tape.gradient(g_loss, generator.trainable_variables)
            g_optimizer.apply_gradients(zip(gradients_g, generator.trainable_variables))

            # ----------- Tambahan step untuk Generator (opsional tuning) -----------
            if d_loss < 0.3 and g_loss > 2.0:
                for _ in range(2):  # Bisa kamu ubah jadi 1 atau 3, sesuai respon model
                    noise_extra = tf.random.normal(shape=(batch_size, latent_dim))
                    with tf.GradientTape() as tape:
                        fake_images = generator(noise_extra, training=True)
                        pred_fake = discriminator(fake_images, training=True)
                        g_loss = loss_fn(tf.ones_like(pred_fake), pred_fake)
                    gradients_g = tape.gradient(g_loss, generator.trainable_variables)
                    g_optimizer.apply_gradients(zip(gradients_g, generator.trainable_variables))

            # Logging per 50 step
            if step % 50 == 0:
                print(f"🔁 Step {step}: D Loss = {d_loss.numpy():.4f}, G Loss = {g_loss.numpy():.4f}")

        print(f"✅ Epoch {epoch+1} selesai: D Loss = {d_loss.numpy():.4f}, G Loss = {g_loss.numpy():.4f}")

        # Optional visualisasi hasil Generator per epoch
        generate_and_plot_images(generator, latent_dim)

        if (epoch + 1) % 10 == 0:
            save_generated_images(generator, latent_dim, epoch)
            save_models(generator, discriminator, epoch)

        with open(os.path.join(SAVE_DIR, 'last_epoch.txt'), 'w') as f:
            f.write(str(epoch + 1))

            zip_output()



    print("\n🎉 Training selesai!")

In [ ]:
latent_dim = 256
g_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5)
d_optimizer = tf.keras.optimizers.Adam(learning_rate=4e-4, beta_1=0.5)
loss_fn = tf.keras.losses.BinaryCrossentropy()
print(f"Total batch per epoch (expected): {len(dataset)}")  # Jumlah batch per epoch



train_gan(generator, discriminator, dataset, latent_dim, g_optimizer, d_optimizer, loss_fn, epochs=300)
